1) Реализовать алгоритм слияния n(n =2**k) отсортированных списков(произвольной длины) в один отсортированный список

In [1]:
import random
import math
import dask
import dask.array as da
import numpy as np
from dask import delayed
import pandas as pd

In [2]:
n=3
lenn = [random.randint(1,16) for i in range(2**n)] # генерирую список, сост из длинн будущих списков в кол-ве n
lst = []
for i in lenn:
    lst.append(sorted([random.randint(0,100) for j in range(i)])) # генерация рандломных числе от 0 до 100 длинной i
print(lst)

[[10, 20, 37, 69], [10, 30, 32, 40, 53, 68, 69, 86, 100], [0, 2, 7, 11, 16, 27, 35, 40, 64, 65, 70, 82], [0, 0, 16, 23, 23, 27, 54, 60, 71, 75, 79, 86, 86, 91], [4, 5, 28, 30, 39, 39, 56, 62, 64, 69, 80, 81, 98], [9, 21, 31, 42, 44, 56, 58, 73, 74, 83, 89], [6, 34, 48, 62, 84], [5, 9, 17, 23, 42, 46, 49, 53, 54, 55, 77, 80, 81, 85, 90, 95]]


In [3]:
def mrg(l1,l2):
    res = []
    itrs = iter(l1), iter(l2)
    vals = [next(itrs[0]),next(itrs[1])]
    while vals != [None, None]:
        cur_i = 0 if vals[0] is not None and (vals[1] is None or vals[0] < vals[1]) else 1
        res.append(vals[cur_i])
        try:
            vals[cur_i] = next(itrs[cur_i])
        except StopIteration:
            vals[cur_i] = None
    return res

In [4]:
def merge(args):
    for i in range(int(math.log(len(args),2))):
        args = [mrg(x, y) for x, y in zip(args[:len(args)//2], args[len(args)//2:])]
    return args

In [5]:
print(merge(lst))

[[0, 0, 0, 2, 4, 5, 5, 6, 7, 9, 9, 10, 10, 11, 16, 16, 17, 20, 21, 23, 23, 23, 27, 27, 28, 30, 30, 31, 32, 34, 35, 37, 39, 39, 40, 40, 42, 42, 44, 46, 48, 49, 53, 53, 54, 54, 55, 56, 56, 58, 60, 62, 62, 64, 64, 65, 68, 69, 69, 69, 70, 71, 73, 74, 75, 77, 79, 80, 80, 81, 81, 82, 83, 84, 85, 86, 86, 86, 89, 90, 91, 95, 98, 100]]


2) С помощью Dask реализовать генерацию n отсортированных списков целых значений (для каждого списка задается собственная длина)
и сохранение каждого списка в отдельный файл (в файле хранится 3 столбц: номерпо_порядку, значение)

In [6]:
def generate_rand(n):
        sett = [da.linspace(np.random.randint(0,10), np.random.randint(20, 50), np.random.randint(10, 20), dtype = 'int', chunks = 1).compute() for i in range(n)]
        return sett

In [7]:
arrays = generate_rand(8)
print(arrays)

[array([ 0,  2,  5,  8, 11, 13, 16, 19, 22, 24, 27, 30, 33]), array([ 5,  6,  8, 10, 12, 13, 15, 17, 19, 20, 22, 24, 26]), array([ 0,  2,  5,  8, 10, 13, 15, 18, 21, 24, 26, 29, 32]), array([ 1,  2,  3,  5,  6,  8,  9, 11, 12, 14, 15, 16, 18, 19, 21, 22, 24,
       25, 26]), array([ 4,  5,  7,  9, 11, 12, 14, 16, 18, 20]), array([ 2,  5,  8, 11, 14, 17, 20, 23, 26, 29, 33]), array([ 0,  2,  5,  7, 10, 12, 15, 17, 20, 23, 25, 28, 30, 33, 35, 38, 40,
       43, 46]), array([ 3,  6,  9, 13, 16, 20, 23, 27, 30, 34, 37, 41, 44, 47])]


3) Применить решение задачи 1 для сортировки данных, сгенерированных в задаче 2

In [8]:
print(merge(arrays))

[[0, 0, 0, 1, 2, 2, 2, 2, 2, 3, 3, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 20, 20, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24, 24, 25, 25, 26, 26, 26, 26, 27, 27, 28, 29, 29, 30, 30, 30, 32, 33, 33, 33, 34, 35, 37, 38, 40, 41, 43, 44, 46, 47]]


4) При помощи dask.delayed распаралелить решение задачи 1 и применить к данным, сгенерированным в задаче 2

In [9]:
def merge_async(args):
    for i in range(int(math.log(len(args),2))):
        args = [delayed(mrg)(x, y) for x, y in zip(args[:len(args)//2], args[len(args)//2:])]
        args = dask.compute(args)[0]
    return args

In [10]:
print(merge_async(arrays))

[[0, 0, 0, 1, 2, 2, 2, 2, 2, 3, 3, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 20, 20, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24, 24, 25, 25, 26, 26, 26, 26, 27, 27, 28, 29, 29, 30, 30, 30, 32, 33, 33, 33, 34, 35, 37, 38, 40, 41, 43, 44, 46, 47]]


5) Модифицировать 4 - обрабатывать данные тем же алгоритмом с помощью pandas.DataFrame

In [11]:
def generate_dataframe(n):
    datalist = []
    for _ in range(2**n):
        datalist.append(pd.DataFrame.from_dict({'just_sum_column' : np.linspace(np.random.randint(0,10), np.random.randint(20, 50), np.random.randint(10, 20), dtype = 'int') }))
    return datalist

In [12]:
data = generate_dataframe(4)
print(data)

[    just_sum_column
0                 5
1                 7
2                 9
3                12
4                14
5                17
6                19
7                22
8                24
9                27
10               29
11               32
12               34
13               37
14               39
15               42
16               44
17               47,     just_sum_column
0                 6
1                 9
2                13
3                17
4                20
5                24
6                28
7                31
8                35
9                39
10               43,     just_sum_column
0                 4
1                 6
2                 8
3                10
4                12
5                15
6                17
7                19
8                21
9                23
10               26,     just_sum_column
0                 9
1                13
2                17
3                21
4                25
5               

6) C помощью разработанного алгоритма и возможностей Dask.DataFrame отсортировать данные, которые обрабатывались на сминаере по Dask.DataFrame (столбцы: ,id,names,amount,new) по столбцу amount.